<style>
@page {
  size: A3 portrait;
  margin: 1cm;
}
body {
  font-size: 12pt;
  margin: 0;
}
</style>

@abdrahym

# ASEAN Cities Weather Data - Data Transfer

[ASEAN Cities Weather Data 2010-YTD(https://www.kaggle.com/datasets/bwandowando/asean-cities-weather-data-2010-2024)](https://www.kaggle.com/datasets/bwandowando/asean-cities-weather-data-2010-2024)

## Initial Setup

In [1]:
# initial setup
import warnings
warnings.filterwarnings("ignore")

import time
%load_ext autoreload
%autoreload 2
time.sleep(1)

%load_ext sql

In [2]:
## get path def

import os
import sys

paths_config = {} 
if os.path.exists(".paths"):
    with open(".paths", 'r') as f:
        for line in f:
            if not line.strip() or '=' not in line:
                continue
            
            key, val = line.strip().split('=')
            key = key.strip()
            path = val.strip().strip("'")

            paths_config[key] = path

## declare paths
target_syspaths = ['gscripts', 'wscripts']
target_varpaths = [ 'datasets', 'dataset1970', 
                    'dataset1980', 'dataset1990', 
                    'dataset2000', 'dataset2010',                    
                    'gscripts', 'wscripts',
                    'gsqls', 'wsqls', 
                    'dbases']

for key, path in paths_config.items():
    if key in target_syspaths:
        if path and path not in sys.path:
            sys.path.append(path)
    
    if key in target_varpaths:
        globals()[key] = path

In [3]:
run_cells = f'{wscripts}/importLibsDb.cellbx.py'
%run -i $run_cells

## DuckDB

### Setup

In [4]:
duckdb_files = f'{dbases}/asean_cities_weather.duckdb'

run_cells = f'{wscripts}/condb_aseancitiesweather_ddb.cellbx.py'
%run -i $run_cells

✅ Successfully connected to DuckDB
   Version: v1.4.3


### explorer object (table)

#### Schema Explorer

In [6]:
qfile = f"{gsqls}/ddb.inspect_schemas.sql"
with open(qfile, 'r') as file:
    qstring = file.read()
ddb_run_dfqstring(qstring,duckdb_files,getmetrict=False)

,catalog_name,schema_name,tables,views,total_objects
0,asean_cities_weather,main,0,5,5
1,asean_cities_weather,models,7,0,7
2,asean_cities_weather,years1970s,5,0,5
3,asean_cities_weather,years1980s,5,0,5
4,asean_cities_weather,years1990s,5,0,5
5,asean_cities_weather,years2000s,5,0,5
6,asean_cities_weather,years2010s,5,0,5


#### 'Main' Schema Explorer

In [7]:
schema_target = 'main'
ddb_inspect_schema(schema_target, duckdb_files)

Found 5 objects in schema 'main'


,Object,Rows,Columns
0,asean_cities,71,5
1,daily_data,"1,433,064",22
2,daily_units,1,21
3,hourly_data,"34,393,536",44
4,hourly_units,1,43


#### 'models' Schema Explorer

In [8]:
schema_target = 'models'
ddb_inspect_schema(schema_target, duckdb_files)

Found 7 objects in schema 'models'


,Object,Rows,Columns
0,hourly_phase_precipitation,"7,165,320",13
1,hourly_phase_soil_condition,"7,165,320",13
2,hourly_phase_solar_radiation,"7,165,320",10
3,hourly_phase_summary,"7,165,320",10
4,hourly_phase_weather_advw,"7,165,320",24
5,hourly_phase_weather_surface,"7,165,320",30
6,hourly_phase_wind,"7,165,320",10


### Transfer Data Mode to pgsql

In [9]:
source_table = 'main.asean_cities'
target_table = 'main.asean_cities'

ddb_to_pgsql(pg_uri=ddb_get_pguri(dbname='aseancities_weather'), 
            source_table=source_table, 
            target_table=target_table, 
            mode='replace',
            db_path=duckdb_files)

Start Transfer Process: 
   main.asean_cities 
     └──> main.asean_cities (replace)
Dropping table pg.main.asean_cities if exists...
---- Uploading main.asean_cities: ----


100%|██████████| 71/71 [00:00<00:00, 1938.72rows/s]

-----------------------------------------------------------------
✅ DuckDB, Single Upload Success
   ddb File : asean_cities_weather.duckdb
   Target   : pg.main.asean_cities
   Source   : 'main.asean_cities'
   Stats    : 71 Rows | 5 Columns | 2.77 KB
   Duration : 2.27s (0m 2s)
-----------------------------------------------------------------

Connection closed.


In [10]:
source_table = 'main.daily_units'
target_table = 'main.daily_units'

ddb_to_pgsql(pg_uri=ddb_get_pguri(dbname='aseancities_weather'), 
            source_table=source_table, 
            target_table=target_table, 
            mode='replace',
            db_path=duckdb_files)

Start Transfer Process: 
   main.daily_units 
     └──> main.daily_units (replace)
Dropping table pg.main.daily_units if exists...
---- Uploading main.daily_units: ----


100%|██████████| 1/1 [00:00<00:00, 35.42rows/s]

-----------------------------------------------------------------
✅ DuckDB, Single Upload Success
   ddb File : asean_cities_weather.duckdb
   Target   : pg.main.daily_units
   Source   : 'main.daily_units'
   Stats    : 1 Rows | 21 Columns | 168.00 B
   Duration : 0.20s (0m 0s)
-----------------------------------------------------------------

Connection closed.


In [11]:
source_table = 'main.hourly_units'
target_table = 'main.hourly_units'

ddb_to_pgsql(pg_uri=ddb_get_pguri(dbname='aseancities_weather'), 
            source_table=source_table, 
            target_table=target_table, 
            mode='replace',
            db_path=duckdb_files)

Start Transfer Process: 
   main.hourly_units 
     └──> main.hourly_units (replace)
Dropping table pg.main.hourly_units if exists...
---- Uploading main.hourly_units: ----


100%|██████████| 1/1 [00:00<00:00, 28.85rows/s]

-----------------------------------------------------------------
✅ DuckDB, Single Upload Success
   ddb File : asean_cities_weather.duckdb
   Target   : pg.main.hourly_units
   Source   : 'main.hourly_units'
   Stats    : 1 Rows | 43 Columns | 344.00 B
   Duration : 0.24s (0m 0s)
-----------------------------------------------------------------

Connection closed.


In [12]:
source_table = 'models.hourly_phase_summary'
target_table = 'models.hourly_phase_summary'

ddb_to_pgsql(pg_uri=ddb_get_pguri(dbname='aseancities_weather'), 
            source_table=source_table, 
            target_table=target_table, 
            mode='replace',
            db_path=duckdb_files)

Start Transfer Process: 
   models.hourly_phase_summary 
     └──> models.hourly_phase_summary (replace)
Dropping table pg.models.hourly_phase_summary if exists...
---- Uploading models.hourly_phase_summary: ----


100%|██████████| 7165320/7165320 [00:31<00:00, 229603.83rows/s]

-----------------------------------------------------------------
✅ DuckDB, Single Upload Success
   ddb File : asean_cities_weather.duckdb
   Target   : pg.models.hourly_phase_summary
   Source   : 'models.hourly_phase_summary'
   Stats    : 7,165,320 Rows | 10 Columns | 546.67 MB
   Duration : 31.43s (0m 31s)
-----------------------------------------------------------------

Connection closed.


In [13]:
source_table = 'models.hourly_phase_weather_advw'
target_table = 'models.hourly_phase_weather_advw'

ddb_to_pgsql(pg_uri=ddb_get_pguri(dbname='aseancities_weather'), 
            source_table=source_table, 
            target_table=target_table, 
            mode='replace',
            db_path=duckdb_files)

Start Transfer Process: 
   models.hourly_phase_weather_advw 
     └──> models.hourly_phase_weather_advw (replace)
Dropping table pg.models.hourly_phase_weather_advw if exists...
---- Uploading models.hourly_phase_weather_advw: ----


100%|██████████| 7165320/7165320 [01:30<00:00, 79038.53rows/s] 

-----------------------------------------------------------------
✅ DuckDB, Single Upload Success
   ddb File : asean_cities_weather.duckdb
   Target   : pg.models.hourly_phase_weather_advw
   Source   : 'models.hourly_phase_weather_advw'
   Stats    : 7,165,320 Rows | 24 Columns | 1.28 GB
   Duration : 91.23s (1m 31s)
-----------------------------------------------------------------

Connection closed.
